# 🎧 Sonarize – Transcrição e Resumo de Áudios do YouTube

Este notebook implementa um pipeline completo para:

1. **Baixar o áudio** de um vídeo do YouTube usando a biblioteca `pytubefix`.
2. **Transcrever o áudio** automaticamente com o modelo Whisper da OpenAI.
3. **Gerar um resumo inteligente** da transcrição utilizando a API do Google Gemini.

## 📌 Funcionalidades

- Download rápido do áudio em formato `.m4a`.
- Transcrição precisa com suporte ao idioma português.
- Resumo automático do conteúdo transcrito com IA generativa.

## 🚀 Como usar

Basta executar as células sequencialmente e fornecer a URL do vídeo quando solicitado.

---

**Observação:** Lembre-se de substituir `"SUA_API_KEY_AQUI"` pela sua chave de API do Google Gemini na terceira célula.

In [ ]:
# Importações
from pytubefix import YouTube
from pytubefix.cli import on_progress

# Entrada do usuário
url = input("🔗 Digite a url: ")

# Criação do objeto YouTube
yt = YouTube(url, on_progress_callback=on_progress)


# Exibição das informações do vídeo
print(f"\n📹 Título: {yt.title}")
print(f"⏱️ Duração: {yt.length} segundos")
print(f"👤 Autor: {yt.author}")

# Download do áudio
print("\n🔄 Iniciando download do áudio...")
ys = yt.streams.get_audio_only()
ys.download(filename="audioyt.m4a")
print('✅ Download concluído!')

In [ ]:
# Importação da biblioteca Whisper (reconhecimento de fala da OpenAI)
import whisper

# Carrega o modelo "base" (equilibrado entre velocidade e precisão)
# Outras opções: tiny, small, medium, large
modelo = whisper.load_model("base")


resultado = modelo.transcribe("audioyt.m4a",   # Transcreve o arquivo de áudio "audioyt.m4a"
                              fp16=False,      # fp16=False: desativa precisão reduzida (compatibilidade)
                              language ="pt")  # language="pt": força o idioma português




with open("transcricao.txt",         # Salva a transcrição em um arquivo de texto
          "w",                       # "w": modo escrita
          encoding="utf-8") as f:    # encoding="utf-8": suporte a caracteres especiais
    # resultado["text"] contém o texto transcrito
    f.write(resultado["text"])

In [ ]:
# Importação da biblioteca Google Gemini AI
from google import genai


# Abre e lê o arquivo de transcrição criado anteriormente
with open('transcricao.txt', 'r') as arquivo:
    conteudo = arquivo.read() # Lê todo o conteúdo do arquivo
    # Cria uma pergunta/prompt para a IA
    pergunta = f"""
Faça um resumo breve do
contedudo: {conteudo}
"""

# Cria cliente com sua chave de API (autenticação)
# Substitua "SUA_API_KEY_AQUI" pela sua chave real
genai.Client(api_key="SUA_API_KEY_AQUI")

# Envia a pergunta para o modelo Gemini
# model='gemini-flash-lite-latest': modelo rápido e leve
# contents=pergunta: o prompt que criamos
response = genai.GenerativeModel('gemini-flash-lite-latest').generate_content(       
     contents=pergunta                       
)


# Extrai e exibe a resposta gerada
resp = response.text                 # Pega o texto da resposta
print(resp)                          # Mostra o resumo no terminal